# Config

In [1]:
import sys

sys.path.append('../pkgs/')

import pytube as pt
import random
import os
import re

In [2]:
import whisper
import subprocess
from urllib.error import HTTPError

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

/Users/dc/anaconda3/envs/fr_whisper/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from pydub import AudioSegment

# Util

## Arguments

In [4]:
random_state = 42

In [5]:
playlist_url = 'https://www.youtube.com/playlist?list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL'
print(playlist_url)

https://www.youtube.com/playlist?list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL


In [6]:
num_videos_download = 10
caption = True

In [14]:
data_dir = '../../../../ForeignWhispers/Data/'

## Functions

In [7]:
def print_playlist_detail(plst):
    print("URL: ", plst.playlist_url)
    print("Playlist Title: ",plst.title)
    print("Playlist Description: ",plst.description)
    print(f"Number of videos in {plst.title} playlist: ", len(plst.video_urls))
    print("Channel: ",plst.owner)

    return

In [8]:
def download_videos(plst, num):
    folder_path = os.path.join(data_dir,'Videos')
    if(not os.path.isdir(folder_path)):
        os.mkdir(folder_path)

    for video in list(plst)[:num]:
        video   = pt.YouTube(video, use_oauth=True, allow_oauth_cache=True)
        print("Downloading Video: ",video.title)
        video.streams.first().download(folder_path)
        print('---------------------')
    return

In [9]:
def download_captions(plst, num):
    folder_path = os.path.join(data_dir,'Captions')
    if(not os.path.isdir(folder_path)):
        os.mkdir(folder_path)
    for video in list(plst)[:num]:
        video   = pt.YouTube(video, use_oauth=True, allow_oauth_cache=True)
        print("Downloading Captions for: ",video.title)
        init = video.streams.first()
        try:
            caption = video.captions['a.en']
            caption.download(title=video.title, output_path= folder_path)
        except KeyError:
            try:
                caption = video.captions['en']
                caption.download(title=video.title, output_path= folder_path)
            except KeyError:
                print("English Caption does not exist!")  
        print('---------------------')
    return

# Downloading videos

In [10]:
playlst = pt.Playlist(playlist_url)
print_playlist_detail(playlst)

URL:  https://www.youtube.com/playlist?list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL
Playlist Title:  Interviews
Playlist Description:  See more at 60Minutes.com
Number of videos in Interviews playlist:  87
Channel:  60 Minutes


In [14]:
download_videos(playlst,num_videos_download)

---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------


In [15]:
download_captions(playlst,num_videos_download)

---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------


# Trancribing videos

## Extracting audio from video

In [14]:
def convert_video_to_audio(data_dir, ar=44100, ac=2, b_a="192k", ext = '.3gpp'):
    '''
    Creating an audio (.mp3) file for every given video file and saving it in the data_dir/Audios folder
    Parameters:
    data_dir: path of the data directory
    ar, ac, b_a: are ffmpeg parameters
    ext: extension of the video files
    '''


    folder_path = os.path.join(data_dir,'Audios')
    if(not os.path.isdir(folder_path)):
        os.mkdir(folder_path)

    for file in os.listdir(os.path.join(data_dir,'Videos')):

        video_file_path = os.path.join(os.path.join(data_dir,'Videos'),file).replace(' ','\ ')
        audio_file_path = os.path.join(os.path.join(data_dir,'Audios'),file.split(ext)[0]).replace(' ','\ ')


        command = f"ffmpeg -i {video_file_path} -vn -ar {ar} -ac {ac} -b:a {b_a} {audio_file_path + '.mp3'}"
        print(command)
        try:
            subprocess.check_output(command, stderr=subprocess.STDOUT, shell=True)
            print(f"Successfully converted {video_file_path} to {audio_file_path}")
        except subprocess.CalledProcessError as e:
            print(f"An error occurred while converting {video_file_path} to {audio_file_path}")
            print(f"Error message: {e.output.decode()}")
            break

# High-quality mono audio - To be run only once
convert_video_to_audio(data_dir, ac=1)

ffmpeg -i ../../../../ForeignWhispers/Data/Videos/World\ Number\ 1\ Pool\ Player\ Shane\ Van\ Boening\ The\ 60\ Minutes\ Interview.3gpp -vn -ar 44100 -ac 1 -b:a 192k ../../../../ForeignWhispers/Data/Audios/World\ Number\ 1\ Pool\ Player\ Shane\ Van\ Boening\ The\ 60\ Minutes\ Interview.mp3


KeyboardInterrupt: 

## Converting audio to text

In [7]:
#initializing model
model = whisper.load_model("base")

In [8]:
%%time

data_dir_ = os.path.join(data_dir,'Audios')

cc_gen = {} #Stores the transcription of every audio file in the data directory

for file in os.listdir(data_dir_):
    if(file.endswith('.mp3')):
        audio = os.path.join(data_dir_,file)

        cc_gen[file.replace('.mp3','')] = model.transcribe(audio)

/Users/dc/anaconda3/envs/fr_whisper/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/dc/anaconda3/envs/fr_whisper/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/dc/anaconda3/envs/fr_whisper/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/dc/anaconda3/envs/fr_whisper/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/dc/anaconda3/envs/fr_whisper/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not 

CPU times: user 44min 9s, sys: 13min 19s, total: 57min 28s
Wall time: 7min 41s


In [9]:
def save_text_file(data_dir, file, text):
    folder_path = os.path.join(data_dir,'Transcript')
    if(not os.path.isdir(folder_path)):
        os.mkdir(folder_path)

    text_file = open(os.path.join(folder_path,f"{file}.txt"), "w")
    text_file.write(text.strip())
    text_file.close()

def save_srt_file(data_dir, file, text):
    folder_path = os.path.join(data_dir,'Transcript')
    if(not os.path.isdir(folder_path)):
        os.mkdir(folder_path)

    srt_file = open(os.path.join(folder_path,f"{file}.srt"), "w")

    for idx_val in text:
        ins_srt = str(idx_val['id']+1)+'\n'
        ins_srt += f"{round(idx_val['start'],2)} --> {round(idx_val['end'],2)}\n"
        ins_srt += idx_val['text'].strip() + "\n"
        ins_srt+='\n'

        srt_file.write(ins_srt)

    srt_file.close()
                
    

def formatting_transcripts(file, val, data_dir,return_srt = False,  save_text = True, save_srt = True):
    '''
    Parameters: 
    file: name of the video
    val: text content of the video generated via whisper
    data_dir: path of the data directory
    return_srt: True if the output of the function be time-stamped text; False for plain text
    save_txt: save the plain text transcription; creates a .txt file in the data_dir/Transcript folder
    save_srt: save the time-stamped text transcription; creates a .srt file in the data_dir/Transcript folder
    '''


    if(save_text):
        save_text_file(data_dir,file,val['text'])
    if(save_srt):
        save_srt_file(data_dir,file,val['segments'])        
    
    if(return_srt):

        return [{k:v for k,v in dic.items() if k in ['id','start','end','text']} for dic in val['segments']]
    
    return val['text'].strip()

        

In [10]:
extracted_txt_arr = []

for name, data in cc_gen.items():
    extracted_txt_arr.append(formatting_transcripts(name, data,  data_dir,False, False, False))

# Translating text to German

In [7]:
lang_dict={'Arabic':'ar','Danish':'da','Filipino':'fil','Finnish':'fi','French':'fr',
           'German':'de','Greek':'el','Gujarati':'gu','Hebrew':'iw',
'Hindi':'hi','Hungarian':'hu','Italian':'it','Japanese':'jap','Russian':'ru','Spanish':'es','Thai':'th','Urdu':'ur'}

In [8]:
class Languages():
    def __init__(self,lang_dictionary):
        self.lang_list = lang_dictionary.keys()
        self.lang_map = lang_dictionary
    
    def is_available(self,lang):
        if(lang not in self.lang_list):
            print("Language not available, please choose one of the languages in the following list: ", *list(self.lang_list))
            return 0
        else:
            print(self.lang_map[lang])
            return 1
    
    def opus_model(self, lang):
        if(self.is_available(lang)):
            model_name =  f"Helsinki-NLP/opus-mt-en-{self.lang_map[lang]}"
            try:
                model = AutoTokenizer.from_pretrained(model_name)
                return model, model_name
            except HTTPError:
                print("Model not available in Opus-MT")
                return None, model_name
    
lang_obj = Languages(lang_dict)
    

In [9]:
def chunkwise_translate(model,sentences,num_sen_per_chunk = 10, device = 'mps'):
    res_sen = []

    for count in range(0,len(sentences),num_sen_per_chunk):
        target_sen = sentences[count:min(count+num_sen_per_chunk,len(sentences))]
        translated = model.generate(**tokenizer(target_sen, return_tensors="pt", padding=True).to(device))
            
        for t in translated:
            res_sen.append(' '.join([_ for _ in [tokenizer.decode(_) for _ in t] if _ not in ['<pad>','</s>','.']]))
        
    return res_sen


def save_translate_srt_file(data_dir, filename, model,language, save = False):
    folder_path = os.path.join(data_dir,'Transcript')
    translate_folder = os.path.join(folder_path,f'translate_{language}')
    
    if(not os.path.isdir(translate_folder)):
        os.mkdir(translate_folder)

    

    with open(os.path.join(folder_path,filename),'r') as file:
        lines = file.readlines()
        file.close()

    line = [_.strip('\n') for _ in lines[2::4]]
    translate_lines = chunkwise_translate(model,line)

    ins_srt = ''

    for idx,l in enumerate(lines):
        if((idx-2)%4==0):
            ins_srt+=translate_lines[idx//4]+'\n'
        else:
            ins_srt += l

    if(save):
        srt_file = open(os.path.join(translate_folder,filename), "w")
        srt_file.write(ins_srt)
        srt_file.close()
        print("Translated file saved at: ", os.path.join(translate_folder,filename))

    return ins_srt

In [10]:
select_lang = 'German'
lang_obj.opus_model(select_lang)

de


(MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-de', vocab_size=58101, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
 	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	58100: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 },
 'Helsinki-NLP/opus-mt-en-de')

In [28]:
tokenizer, model_name = lang_obj.opus_model(select_lang)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name,device_map='mps')

de


In [19]:
# Sample srt file for one video
translated_srt = save_translate_srt_file(data_dir,'Attorney General Merrick Garland The 60 Minutes Interview.srt',model, language= select_lang, save = True)

Translated file saved at:  ../../../../ForeignWhispers/Data/Transcript/translate_German/Attorney General Merrick Garland The 60 Minutes Interview.srt


In [29]:
file_to_translate = []
for file in os.listdir('../../../../ForeignWhispers/Data/Transcript'):
    if(file.endswith('.srt')):
        file_to_translate.append(file)
print(file_to_translate)
print(len(file_to_translate))

['Rich Paul The 60 Minutes Interview.srt', 'Volodymyr Zelenskyy The 2023 60 Minutes Interview.srt', 'Pink The 60 Minutes Interview.srt', 'Godfather of AI Geoffrey Hinton The 60 Minutes Interview.srt', 'President Joe Biden The 2023 60 Minutes Interview.srt', 'Deion Sanders The 2023 60 Minutes Interview.srt', 'Gen Mark Milley The 60 Minutes Interview.srt', 'Attorney General Merrick Garland The 60 Minutes Interview.srt', 'World Number 1 Pool Player Shane Van Boening The 60 Minutes Interview.srt', 'Denzel Washington  60 Minutes Archive.srt']
10


In [31]:
#Saving translated srts of all audios

for file in file_to_translate:
    print(file)
    translated_srt = save_translate_srt_file(data_dir,file,model, language=select_lang,save = True)

Rich Paul The 60 Minutes Interview.srt
Translated file saved at:  ../../../../ForeignWhispers/Data/Transcript/translate_German/Rich Paul The 60 Minutes Interview.srt
Volodymyr Zelenskyy The 2023 60 Minutes Interview.srt
Translated file saved at:  ../../../../ForeignWhispers/Data/Transcript/translate_German/Volodymyr Zelenskyy The 2023 60 Minutes Interview.srt
Pink The 60 Minutes Interview.srt
Translated file saved at:  ../../../../ForeignWhispers/Data/Transcript/translate_German/Pink The 60 Minutes Interview.srt
Godfather of AI Geoffrey Hinton The 60 Minutes Interview.srt
Translated file saved at:  ../../../../ForeignWhispers/Data/Transcript/translate_German/Godfather of AI Geoffrey Hinton The 60 Minutes Interview.srt
President Joe Biden The 2023 60 Minutes Interview.srt
Translated file saved at:  ../../../../ForeignWhispers/Data/Transcript/translate_German/President Joe Biden The 2023 60 Minutes Interview.srt
Deion Sanders The 2023 60 Minutes Interview.srt
Translated file saved at:  .

# Text to Speech german

In [11]:
def read_srt(srt_filepath):
    '''Reads a srt file and returns a dictionary of dictionaries.
    idx is the key, value is a dictionary - containing timestamped sentences'''
    srt_dic = {}

    with open(srt_filepath,'r') as file:
        lines = file.readlines()
        file.close()
    
    block_dic = {}

    for idx,line in enumerate(lines):
        if(line == '\n'):
            srt_dic[dic_key] = block_dic
            block_dic = {}
        if(idx%4==0):
            dic_key = line.strip('\n')
        elif(idx%4==1):
            times = line.strip('\n ').split(' ')
            block_dic['start_time'],block_dic['end_time'] = times[0],times[-1]
        elif(idx%4==2):
            statement = line.strip('\n')
            block_dic['statement'] = statement
        else:
            continue
    
    return srt_dic


def generate_audio(data_dir,srt_filepath,language):
    '''Generated audio for the given srt file.
    One block of srt is converted to speech at a time. 
    We add pause time if the time of translation is lower than the time of srt block
    Saves the output in a wav file

    data_dir: directory of containing the raw data
    srt_filepath: path of the translated srt
    language: name of the target translation language
    '''
    translate_audio_folder = os.path.join(os.path.join(data_dir,'Audios'),f'translate_{language}')
    # audio_folder = os.path.join(data_dir,'Audios')
    audio_name = srt_filepath.split('/')[-1].replace('.srt','')
    
    if(not os.path.isdir(translate_audio_folder)):
        os.mkdir(translate_audio_folder)
        print("Folder Created: ", translate_audio_folder)
    else:
        print("Folder already present!")
    
    if(not os.path.exists(os.path.join(translate_audio_folder,audio_name) + '.wav')):

    
        srt_out = read_srt(srt_filepath)

        audio_sum = AudioSegment.empty()

        first_sub = 1

        for idx,sub in srt_out.items():
            print(idx)
            if(len(sub['statement'].strip())>0):
                temp = subprocess.call(['tts', '--text', sub['statement'],'--model_name', 'tts_models/de/thorsten/vits',
                                        '--speed','1.5','--out_path', 'temp.wav'])
                
                sub_audio = AudioSegment.from_wav('temp.wav')

                if first_sub:
                    silence_time = float(sub['start_time'])*1000
                else:
                    silence_time = (float(sub['end_time']) - float(sub['start_time']))*1000 - len(sub_audio)


                audio_sum = audio_sum + sub_audio + AudioSegment.silent(duration=silence_time)

                first_sub = 0
                os.remove('temp.wav')

        # orig_length = len(AudioSegment.from_mp3(f'{audio_folder}/{audio_name}.mp3'))
        # new_length = len(audio_sum)
        
        with open(os.path.join(translate_audio_folder,audio_name) + '.wav', 'wb') as out_f:
            audio_sum.export(out_f, format='wav',) 
    else:
        print('TTS already done!')
        
    return

In [12]:
audio_srts = []
for file in os.listdir(f'../../../../ForeignWhispers/Data/Transcript/translate_{select_lang}'):
    if(file.endswith('.srt')):
        audio_srts.append(file)
print(audio_srts)
print(len(audio_srts))

['Rich Paul The 60 Minutes Interview.srt', 'Volodymyr Zelenskyy The 2023 60 Minutes Interview.srt', 'Pink The 60 Minutes Interview.srt', 'Godfather of AI Geoffrey Hinton The 60 Minutes Interview.srt', 'President Joe Biden The 2023 60 Minutes Interview.srt', 'Deion Sanders The 2023 60 Minutes Interview.srt', 'Gen Mark Milley The 60 Minutes Interview.srt', 'Attorney General Merrick Garland The 60 Minutes Interview.srt', 'World Number 1 Pool Player Shane Van Boening The 60 Minutes Interview.srt', 'Denzel Washington  60 Minutes Archive.srt']
10


In [16]:
#Showing TTS for one srt
for file in audio_srts[6:7]:
    srt_file = os.path.join(f'../../../../ForeignWhispers/Data/Transcript/translate_{select_lang}',file)
    print(srt_file)
    generate_audio(data_dir,srt_file,select_lang)

../../../../ForeignWhispers/Data/Transcript/translate_German/Gen Mark Milley The 60 Minutes Interview.srt
Folder already present!
1
 > tts_models/de/thorsten/vits is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Text: General Mark Mil ley beendet e eine vie